# Análise estatística do dataset "World COVID-19 Data"
https://www.kaggle.com/datasets/abhishek14398/world-covid19-data

Arquivos utilizados:
- CONVENIENT_global_confirmed_cases.csv
    - Primeira linha: lista de países/regiões
    - Segunda linha: lista de províncias/estados para alguns países apenas
    - Primeira coluna: datas em dias
    - Colunas seguintes, quantidade de casos confirmados
- CONVENIENT_global_deaths.csv
    - Primeira linha: lista de países/regiões
    - Segunda linha: lista de províncias/estados para alguns países apenas
    - Primeira coluna: datas em dias
    - Colunas seguintes, quantidade de casos de mortes
- CONVENIENT_global_metadata.csv
    - Primeira linha: header
    - Primeira coluna: índice
    - Segunda coluna: países/regiões
    - Terceira coluna: províncias/estados
    - Quarta coluna: latitude
    - Quinta coluna: longitude

In [ ]:
# instalação de ffmpeg:
# !pip install ffmpeg-python
# from IPython.display import HTML
# Necessita do ffmpeg, então optamos por não utilizar
# Para utilizar o plotly com regressão linear, necessita instalar statsmodel
# %pip install statsmodels
# importando pacotes
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import plotly.express as px
import plotly.graph_objects as go

# ignorar warnings (limpar notebook)
warnings.filterwarnings('ignore')

# constantes
DATA_FOLDER = './data'

CONFIRMED_CASES_DATA = 'CONVENIENT_global_confirmed_cases.csv'
DEATH_CASES_DATA = 'CONVENIENT_global_deaths.csv'
LOCATION_DATA = 'CONVENIENT_global_metadata.csv'

In [ ]:
# realizando a leitura dos dados
# esses arquivos tem duas linhas de header, onde a primeira é o nome do país
# a segunda é uma região do país, caso exista, senão, vazio (NaN)
path_ = os.path.join(DATA_FOLDER, DEATH_CASES_DATA)
death = pd.read_csv(path_, sep=',', header=[0,1])

# agregando os valores para o mesmo país (somando)
# dado que os países eram os nomes das colunas,
# fazer transposição para poder agregar por nome do país
death = death.T
# reseta o index para que os index virem colunas
death.reset_index(inplace=True)
# modifica o nome das colunas para as datas
death.columns = death.loc[0]
# dropa os nomes das colunas usadas acima
death.drop([0], axis=0, inplace=True)
# dropa a coluna da província, dado que vamos agrupar posteriormente por país
death.drop(['Province/State'], inplace=True, axis=1)
# agrupa por país somando os valores de casos
death = death.groupby("Country/Region").sum()
# faz a transposta novamente para retornar ao formato original
death = death.T
# joga os index como colunas
death.reset_index(inplace=True)
# renomeia a coluna 0 antigo índice para data
death.rename(columns={0: 'data'}, inplace=True)
# remove o nome das colunas que fica aparecendo em cima do novo index
death.columns.names = ['']
# transformando a coluna Data no tipo Data
death.data = pd.to_datetime(death.data)

# idem para confirmed
path_ = os.path.join(DATA_FOLDER, CONFIRMED_CASES_DATA)
confirmed = pd.read_csv(path_, sep=',', header=[0,1])
confirmed = confirmed.T
confirmed.reset_index(inplace=True)
confirmed.columns = confirmed.loc[0]
confirmed.drop([0], axis=0, inplace=True)
confirmed.drop(['Province/State'], inplace=True, axis=1)
confirmed = confirmed.groupby("Country/Region").sum()
confirmed = confirmed.T
confirmed.reset_index(inplace=True)
confirmed.rename(columns={0: 'data'}, inplace=True)
confirmed.columns.names = ['']
confirmed.data = pd.to_datetime(confirmed.data)

# arquivo de latitude e longitude
# primeira coluna de indice
path_ = os.path.join(DATA_FOLDER, LOCATION_DATA)
location = pd.read_csv(path_, sep=',', index_col=0)
# no caso do location tem que criar uma nova coluna para match os nomes dos headers dos dfs death e confirmed
#location["Region"] = location["Country/Region"].str.cat(location["Province/State"], sep="_", na_rep="")
# remover _ de células onde o Province/State era NaN
#location["Region"] = location["Region"].str.removesuffix("_")


In [ ]:
death

In [ ]:
confirmed

In [ ]:
location
# embora haja muitos valores NaN, nao podemos remover as linhas com NaN, podemos substituir, mas nao sera necessario
# pois sera feito posteriormente
# checar qtd de NaN e qtd em fração de Nan

In [ ]:
# checando valores nulos
print(sum(death.isna().sum())) #não há valores nulos
print(sum(confirmed.isna().sum())) #não há valores nulos
print(location.isna().sum()) #há valores nulos

In [ ]:
# verificando coluna a coluna o que são os valores nulos
display(location[location['Lat'].isna()]) #dois países com regiões não utilizados
display(location[location['Long'].isna()]) #dois países com regiões não utilizados
location.dropna(subset=['Long', 'Lat'], inplace=True)
# não usaremos a coluna Province/States
location.drop('Province/State', axis=1, inplace=True)


In [ ]:
# agrupando location pelo pais:
location = location.groupby('Country/Region').first()
#location[location['Country/Region'] == "Australia"]

In [ ]:
# confirmando se o shape de death e confirmed são iguais (mesma qtd de linhas e colunas)
print(death.shape == confirmed.shape)
# confirmando se as datas são iguais analisando uma a uma, somando os trues e comparando a quantidade de datas
print(sum(death.data == confirmed.data) == death.shape[0])
# confirmando se os nomes das colunas são iguais
print(sum(death.columns == confirmed.columns) == (death.columns.size))
# confirmando se o location["Regions"] bate com os nomes das colunas dos outros dois dfs:
print(sum(location.index == death.columns[1:]) == death.columns[1:].size)

In [ ]:
# verificando informações do DF
death.info

In [ ]:
# analise estatistica
stats_death = death.describe().T
stats_death
# possivelmente dados negativos são retratações de dados informados anteriormente

In [ ]:
# análise estatística
stats_confirmed = confirmed.describe().T
stats_confirmed

In [ ]:
# eliminar variáveis de variância nula (abaixo de um limiar)
stats_death['CV'] = abs(stats_death['std'] * 100 / stats_death['mean'])
stats_confirmed['CV'] = abs(stats_confirmed['std'] * 100 / stats_confirmed['mean'])
display(stats_death)
display(stats_confirmed)
# nao faz sentido este tratamento para este dataset.

In [ ]:
# checando paises com numero de mortes menores que dois
stats_death[stats_death['max'] < 2]

In [ ]:
# criando novo DF agrupando por ano e mes e somando os casos de mortes
death_monthly = death.groupby([death.data.dt.year, death.data.dt.month])[death.columns[1:]].sum()
# fazendo a soma cumulativa dos casos
death_monthly = death_monthly.cumsum()
# renomeando o indice
death_monthly.index.rename(['ano', 'mes'], inplace=True)
# resetando o indice para transformar em colunas do DF
death_monthly.reset_index(inplace=True)
# criando uma nova coluna de data unindo ano e mes e adicionando leading zero
death_monthly["data"] = death_monthly["ano"].astype(str) + death_monthly["mes"].astype(str).str.zfill(2)
death_monthly

In [ ]:
# idem para confirmed
confirmed_monthly = confirmed.groupby([confirmed.data.dt.year, confirmed.data.dt.month])[confirmed.columns[1:]].sum()
confirmed_monthly = confirmed_monthly.cumsum()
confirmed_monthly.index.rename(['ano', 'mes'], inplace=True)
confirmed_monthly.reset_index(inplace=True)
confirmed_monthly["data"] = confirmed_monthly["ano"].astype(str) + confirmed_monthly["mes"].astype(str).str.zfill(2)
confirmed_monthly

In [ ]:
%%time
# criando novo dataframe com o formato apropriado para utilizar plotly.express.scatter_geo:
# data - confirmed - pais - lat - lon
# para utilizar com scatter_geo
novo_df = pd.DataFrame(columns=['data', 'confirmed', 'death', 'pais', 'lat', 'lon'])
# itera pelas colunas com nomes dos países
for coluna in confirmed_monthly.columns[2:-1]:
    # cria um dataframe temporário com a data e o país
    temp = confirmed_monthly[['data', coluna]]
    # cria a coluna de mortes buscando no dataframe das mortes
    temp["death"] = death_monthly[[coluna]]
    temp['pais'] = temp.columns[1]
    temp['lat'] = location.loc[coluna]['Lat']
    temp['lon'] = location.loc[coluna]['Long']
    temp.columns = ['data', 'confirmed', 'death', 'pais', 'lat', 'lon']
    novo_df = pd.concat([novo_df, temp], ignore_index=True)
novo_df

In [ ]:
# criando novo dataframe com o formato apropriado para utilizar plotly.express.scatter_geo:
# data - confirmed - pais - lat - lon
# para utilizar com scatter_geo
# agora utilizando apply
# mudando o index para utilizar a data
confirmed_monthly.index = confirmed_monthly['data']
# removendo colunas que não serão usadas no apply
confirmed_monthly.drop(columns=['data', 'ano', 'mes'], inplace=True)

In [ ]:
# definindo novo dataframe resultado
novo_df2 = [pd.DataFrame()]
# definindo função para ser usado no apply
def cria_df(coluna, df_externo):
    temp = pd.DataFrame()
    temp['data'] = coluna.index
    temp['confirmed'] = coluna.values
    temp['death'] = death_monthly[coluna.name].values
    temp['pais'] = coluna.name
    temp['lat'] = location.loc[coluna.name]['Lat']
    temp['lon'] = location.loc[coluna.name]['Long']
    df_externo[0] = pd.concat([df_externo[0], temp], ignore_index=True)
    return coluna
# por que precisa usar lista? Questão de referência, seria possível fazer sem lista?

In [ ]:
%%time
# utilizando o apply
confirmed_monthly.apply(cria_df, df_externo=novo_df2)
novo_df2 = novo_df2[0]

In [ ]:
# comparando os dataframes
display(novo_df)
print("="*20)
display(novo_df2)

In [ ]:
# checando se os dataframes resultantes são iguais:
print((novo_df == novo_df2).sum())
print(novo_df.shape)

In [ ]:
# plotando utilizando o plotly e scatter_geo
fig = px.scatter_geo(
    novo_df, # dataframe utilizado para plotar
    locations="pais", # campo utilizado na localização
    locationmode='country names', # como location foi utilizado
    color="confirmed", # escala de cores
    size="confirmed", # tamanho dos círculos
    hover_name="pais", # informação a aparecer ao passar o mouse
    range_color= [0, 50000000], # range de cores para o heatmap
    projection="natural earth", # projeção do globo
    animation_frame="data", # dado utilizado como eixo para animação
    color_continuous_scale="portland",
    opacity=0.7,
    title='Casos confirmados de COVID-19 por país', # título
    width=1000,
    height=500
    )
fig.update_geos(visible=True, resolution=50, scope="world", showcountries=True, countrycolor="gray")
fig.show()

In [ ]:
fig = px.scatter_geo(
    novo_df,
    locations="pais",
    locationmode='country names',
    color="death",
    size="death",
    hover_name="pais",
    range_color= [0, 500000],
    projection="natural earth",
    animation_frame="data",
    color_continuous_scale="portland",
    opacity=0.7,
    title='Mortes de COVID-19 por país',
    width=1000,
    height=500
    )
fig.update_geos(visible=True, resolution=50, scope="world", showcountries=True, countrycolor="gray")
fig.show()

In [ ]:
# plotando grafico de linhas com os valores acumulados por pais
fig = px.line(novo_df, x="data", y="confirmed", color='pais', width=1000, height=500)
fig.show()

In [ ]:
# plotando grafico de linhas com os valores acumulados por pais
fig = px.line(novo_df, x="data", y="death", color='pais', width=1000, height=500)
fig.show()

In [ ]:
# Pergunta: Qual o máximo de casos confirmados diariamente de covid? Qual o país?
#            e para mortes?
print(confirmed.max(numeric_only=True).sort_values(ascending=False)) #US foi o maior caso
print("="*20)
print(death.max(numeric_only=True).sort_values(ascending=False))



In [ ]:
# Pergunta: Quando houve este pico máximo?
print(confirmed[confirmed["US"] == confirmed.max(numeric_only=True).max()]["data"]) #2022-01-10
print("="*20)
print(death[death["Chile"] == death.max(numeric_only=True).max()]["data"]) #2022-03-21

In [ ]:
# Pergunta: Será que houve algum "erro" de informação onde acumulou muito caso em um dia?
display(confirmed[(confirmed["data"] > "2022-01-05") & (confirmed["data"] < "2022-01-15")][["data", "US"]])
print("="*20)
display(death[(death["data"] > "2022-03-16") & (death["data"] < "2022-03-26")][["data", "Chile"]])


In [ ]:
# Consideração: O dado do Chile aparenta estar com algum erro, o dado está destoante do restante
fig = px.line(death, x="data", y="Chile", width=1000, height=500)
fig.show()
# Confirmando online, verificou-se que nesta data teve esta quantidade de mortes no Chile.

In [ ]:
# Pergunta: Qual país teve a menor variância no número de casos confirmados e de morte?
display(stats_confirmed[stats_confirmed["CV"] == stats_confirmed["CV"].min()])
display(stats_death[stats_death["CV"] == stats_death["CV"].min()])

In [ ]:
# criando um novo dataframe concatenando as séries com a somatória das mortes e casos confirmados para cada país
# utilize apply para criar uma nova coluna com a taxa de mortalidade
mortalidade = pd.concat([death.sum(), confirmed.sum()], axis=1)
mortalidade.reset_index(inplace=True)
mortalidade.columns = ['country', 'death', 'confirmed']
mortalidade['taxa_mortalidade'] = mortalidade.apply(lambda x: x['death']/x['confirmed'], axis=1)
mortalidade

In [ ]:
# Pergunta: Qual o país que teve maior taxa de mortalidade para casos confirmados?
mortalidade.sort_values(by='taxa_mortalidade', ascending=False, inplace=True)
mortalidade.head(60)
# Verifica-se que os dados da Korea do Norte não fazem sentidos, tem um total de 6 mortes e um total de 1 caso confirmado
# Verifica-se também que a maioria dos países que tiveram uma maior taxa de mortalidade são países pobres, com exceção do Peru, México e Equador

In [ ]:
# removendo dados de morte = 0 para calcular a trendline
mortalidade = mortalidade[mortalidade["death"] > 0]

In [ ]:
# Pergunta: Qual o país que teve menor taxa de mortalidade para casos confirmados?
fig = px.scatter(mortalidade, x="confirmed", y="death", hover_name="country", width=800, height=400, log_x=True, log_y=True,
                 trendline="ols", trendline_color_override="red", trendline_scope="overall", trendline_options=dict(log_x=True, log_y=True))
fig.show()

In [ ]:
#fazendo copia do dataframe para não alterar o original
mortalidade2 = mortalidade.copy()
# removendo os países com poucos casos confirmados
mortalidade2 = mortalidade2[mortalidade2["confirmed"] > 1000]
# removendo os países com poucas mortes
mortalidade2 = mortalidade2[mortalidade2["death"] > 100]
# plotando mortes por casos confirmados com grafico scatter logaritmico
fig = px.scatter(mortalidade2, x="confirmed", y="death", hover_name="country", width=800, height=400, log_x=True, log_y=True, trendline="ols", trendline_color_override="red", trendline_scope="overall", trendline_options=dict(log_x=True, log_y=True))
fig.show()